# Loading data

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# DATA fusion
Code for fusion of data sensors considering the imputation, plots and metrics calculus for the fusion and the single sensors. 

RMSE, the F1 score and accuracy are calulated.

The F1 score is saved in the file 'F1values_fusion_new.csv'' in the selected folder.

To set desired experiments, filters, sensors and options, use next lines of code:

In [ ]:
#Set option

#Set sensors: set sensors to consider by setting their variable to 'True'. For ITSC please use the configuration: range_sens='False', cam_w_yolo_FT='True', cam_d_yolo_FT='True', cam_w_yolo_v5='True', cam_d_yolo_v5='True'
range_sens='False' 
cam_w_yolo_FT='True'
cam_d_yolo_FT='True'
cam_w_yolo_v5='True'
cam_d_yolo_v5='True'

#Set experiments between 0 and 13. Multiple experiments can be set.
#set experiments of scene A or B. Scene A includes experiments from 0 to 5, while scene B from 6 to 11
scena='B'

if scena=='A':
    exp_list=[0,1,2,3,4,5]
elif scena=='B':
    exp_list=[6,7,8,9,10,11]

#set filters: between 'original','bright_1','bright_2','bright_3','dark_1','dark_2','dark_3','fog_1','fog_2','fog_3','rain_1','rain_2','rain_3'. Multiple filters can be set:
filter_list=['original','bright_1','bright_2','bright_3','dark_1','dark_2','dark_3','fog_1','fog_2','fog_3','rain_1','rain_2','rain_3']

#set folder name where the dataset is located
folder= "safe_road_data"

#set set_plot= 'True' if you want to see the plots:
set_plot= 'True'

print("exp_list="+ str(exp_list))
print("filter_list="+ str(filter_list))
print("folder="+ str(folder))

# Danger function definition

In [ ]:
# Distance component
def d_f(distance):
    if distance <= 0:
        return 0
    return 1 / math.log(distance + 0.6)

# Speed component
def d_s(speed):
    s_min = 0.05
    s_max = 0.5
    if speed <= s_min:
        return 0
    if speed >= s_max:
        return 1
    return (speed-s_min)/(s_max - s_min)

#Acceleration component
def d_a(acceleration):
    a_min = 1
    a_max = 10
    if acceleration <= - a_max:
        return -1
    elif -a_max < acceleration <= -a_min:
        return (acceleration+a_min)/(a_max - a_min)
    elif -a_min < acceleration <= +a_min:
        return 0
    elif a_min < acceleration <= +a_max:
        return (acceleration - a_min)/(a_max - a_min)
    elif acceleration > a_max:
        return 1
    return 0
    #if acceleration < 1 and acceleration > -1: # -1 < a < 1
        #return 1
    #elif acceleration < -1:
        #return 0.9
    #else:
        #return 1.1

# for plotting purposes returns just the distance component of the danger function
def danger_function_distance(row, sensor ,k1 = 1, k2 = 1, k3 = 1):
    distance = row[f'distance']+ 0.000001
    speed = row[f'speed']
    acceleration = row[f'acceleration']
    distance_factor = d_f(distance)
    speed_factor = d_s(speed)
    acceleration_factor = d_a(acceleration)
    return distance_factor

# for plotting purposes returns just the speed component of the danger function
def danger_function_speed(row, sensor ,k1 = 1, k2 = 1, k3 = 1):
    distance = row[f'distance']+ 0.000001
    speed = row[f'speed']
    acceleration = row[f'acceleration']
    distance_factor = d_f(distance)
    speed_factor = d_s(speed)
    acceleration_factor = d_a(acceleration) 
    return speed_factor

# for plotting purposes returns just the acceleration component of the danger function
def danger_function_acceleration(row, sensor ,k1 = 1, k2 = 1, k3 = 1):
    distance = row[f'distance']+ 0.000001
    speed = row[f'speed']
    acceleration = row[f'acceleration']
    distance_factor = d_f(distance)
    speed_factor = d_s(speed)
    acceleration_factor = d_a(acceleration)
    return acceleration_factor

# complete danger function, computes instant danger as combination of pre defined component
def danger_function_composed(row,sensor ,k1 = 0.1):
    distance = row[f'distance_{sensor}'] + 0.000001
    speed = row[f'speed_{sensor}']
    acceleration = row[f'acceleration_{sensor}']
    distance_factor = d_f(distance)
    speed_factor = d_s(speed)
    acceleration_factor = d_a(acceleration)
    return (speed_factor + k1 * acceleration_factor) * distance_factor

In [ ]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
pd.reset_option('all')

In [ ]:
F1values=pd.DataFrame()
RMSEvalues=pd.DataFrame()
ACCvalues=pd.DataFrame()

for filter in filter_list:
    exec(f"RMSE_{filter}= pd.DataFrame()")
    exec(f"F1_score_{filter}= pd.DataFrame()")
    exec(f"Accuracy_score_{filter}= pd.DataFrame()")
    
    exec(f"RMSE_{filter}_mean= pd.DataFrame()")
    exec(f"RMSE_{filter}_std= pd.DataFrame()")
    exec(f"Accuracy_score_{filter}_std= pd.DataFrame()")
    exec(f"F1_score_{filter}_mean= pd.DataFrame()")
    exec(f"F1_score_{filter}_std= pd.DataFrame()")
    exec(f"Accuracy_score_{filter}_std= pd.DataFrame()")


    for exp_nr in exp_list:
    
        #older_name= f"{folder}/exp_{exp_nr}/pre_fusion_data"
        #set folder name
        folder_name= f"{folder}/exp_{exp_nr}/pre_fusion_data_forfusion"
        df_100_pre = pd.read_csv(f'{folder_name}/pre_fusion_data_{filter}.csv')
        if df_100_pre.index[0]==0.00:
            df_100_pre=df_100_pre.drop(0.00, axis='index')
        #print(f'ok1 {exp_nr}')
        df_100_pre.index = df_100_pre['Unnamed: 0']
        df_100_pre.rename_axis('timestamp', inplace=True)
        df_100_pre.drop(['Unnamed: 0'], axis=1, inplace=True)
        #print(f'ok2 {exp_nr}')
        # Distance Fusion
        ## Distance values fusion
        distance_fusion=pd.DataFrame()
        data=pd.DataFrame()
        data=df_100_pre["wheelchair_data_FT"].copy(deep='True')
        if cam_w_yolo_FT=='True':
            distance_fusion = pd.concat([distance_fusion,df_100_pre["distance_wheelchair_FT"]], axis=1)
            data=(data|df_100_pre["wheelchair_data_FT"])
        if cam_w_yolo_v5=='True':
            distance_fusion = pd.concat([distance_fusion,df_100_pre["distance_wheelchair_v5"]], axis=1)  
            data=(data|df_100_pre["wheelchair_data_v5"])  
        if cam_d_yolo_FT=='True':
            distance_fusion = pd.concat([distance_fusion,df_100_pre["distance_drone_FT"]], axis=1)    
            data=(data|df_100_pre["drone_data_FT"])
        if cam_d_yolo_v5=='True':
            distance_fusion = pd.concat([distance_fusion,df_100_pre["distance_drone_v5"]], axis=1)    
            data=(data|df_100_pre["drone_data_v5"])
        if range_sens=='True':
            distance_fusion = pd.concat([distance_fusion,df_100_pre["distance_range"]], axis=1)    
            data=(data|df_100_pre["range_data"])
            
        df_100_pre['distance_fusion'] = distance_fusion.mean(axis=1)


        # Distance Fusion
        #cosa significa? l'ho fatto bene? confronto
      
        data=data.astype(int)
        dist = df_100_pre['distance_fusion']*data
        dist = dist.replace(0, np.nan)
        

        #stamps = df_100_pre.index*(df_100_pre['wheelchair_data'] | df_100_pre['drone_data'] | df_100_pre['range_data']).astype(int)
        stamps = df_100_pre.index*data
        stamps = stamps.replace(0, np.nan)

        ## Speed values fusion
        df_speed = pd.DataFrame(dist, index=stamps, columns=['distance_fusion'])
        #df_speed=pd.DataFrame()
        df_speed = df_speed.dropna()        
        df_speed['speed_fusion'] = np.insert(- np.diff(df_speed['distance_fusion'])/np.diff(df_speed.index), 0, 0)
        max_speed = 3.5
        min_speed = 0 # We assume the obstacle is not going backwards
        df_speed['speed_fusion'] = np.clip(df_speed['speed_fusion'], a_min=min_speed, a_max=max_speed)
        df_100_pre['speed_fusion'] = df_speed['speed_fusion'].copy(deep='True')
        df_100_pre['speed_fusion'] = df_100_pre['speed_fusion'].ffill()

        ## Acceleration values fusion
        df_speed['acceleration_fusion'] = np.insert(- np.diff(df_speed['speed_fusion'])/np.diff(df_speed.index), 0, 0)
        df_100_pre['acceleration_fusion'] = df_speed['acceleration_fusion']
        df_100_pre['acceleration_fusion'] = df_100_pre['acceleration_fusion'].ffill()

        df_100_pre['speed_fusion'] = df_100_pre['speed_fusion'].fillna(0)
        df_100_pre['acceleration_fusion'] = df_100_pre['acceleration_fusion'].fillna(0)
        

        ## Danger function application on distance fusion and tracker danger computation
        df_100_pre['danger_fusion'] = df_100_pre.apply(lambda row: danger_function_composed(row, 'fusion'), axis=1)
        df_100_pre['danger_tracker'] = df_100_pre.apply(lambda row: danger_function_composed(row, 'tracker'), axis=1)
        df_100_pre['danger_tracker'] =df_100_pre['danger_tracker'].fillna(0)
        df_dist_noroll=df_100_pre['danger_tracker'].copy(deep='True')
        df_100_pre['danger_fusion'] = df_100_pre['danger_fusion'].fillna(0)
 
        folder_name= f"{folder}/exp_{exp_nr}/pre_fusion_data"
        #set folder name
        df_100 = pd.read_csv(f'{folder_name}/pre_fusion_data_{filter}.csv')
        if df_100.index[0]==0.00:
            df_100=df_100.drop(0.00, axis='index')
        #print(f'ok1 {exp_nr}')
        df_100.index = df_100['Unnamed: 0']
        df_100.rename_axis('timestamp', inplace=True)
        df_100.drop(['Unnamed: 0'], axis=1, inplace=True)

        df_100['danger_tracker'] = df_100.apply(lambda row: danger_function_composed(row, 'tracker'), axis=1)
        df_100['danger_tracker'] =df_100['danger_tracker'].fillna(0)
        

        #df_100=pd.concat([df_100,df_100_fusion])
        df_100['danger_fusion']=df_100_pre['danger_fusion'].copy(deep='True')
        df_100['distance_fusion']=df_100_pre['distance_fusion'].copy(deep='True')
        df_100['speed_fusion']=df_100_pre['speed_fusion'].copy(deep='True')
        df_100['acceleration_fusion']=df_100_pre['acceleration_fusion'].copy(deep='True')
  
        df_100['distance_fusion']=df_100['distance_fusion'].fillna(4)
        df_100['danger_fusion']=df_100['danger_fusion'].fillna(0)
        df_100['speed_fusion']=df_100['speed_fusion'].fillna(0)
        df_100['acceleration_fusion']=df_100['acceleration_fusion'].fillna(0)
        df_100['danger_fusion_smooth'] = df_100['danger_fusion'].rolling(window=10).mean()
        df_100['danger_fusion_smooth']=df_100['danger_fusion_smooth'].fillna(0)

        #df_dist_noroll=df_100['danger_fusion'].copy(deep='True')

        
        if set_plot=='True':
            #plot start---------------------_
            fig, ax = plt.subplots(3,1)
            # Index 4 Axes arrays in 4 subplots within 1 Figure: 
            ax[0].title.set_text(f'Distance comparison exp {exp_nr} with filter {filter}')
            ax[0].plot(df_100.index, df_100['distance_tracker'], label='track')
            ax[0].plot(df_100.index, df_100['distance_fusion'], label='fus')

            # if cam_d_yolo_FT=='True':
            #     ax[0].plot(df_100.index, df_100['distance_drone_FT'], label='d_FT')
            # if cam_d_yolo_v5=='True':
            #     ax[0].plot(df_100.index, df_100['distance_drone_v5'], label='d_v5')#,marker='o', linestyle=' ')
            # if cam_w_yolo_FT=='True':
            #     ax[0].plot(df_100.index, df_100['distance_wheelchair_FT'], label='w_FT')
            # if cam_w_yolo_v5=='True':
            #     ax[0].plot(df_100.index, df_100['distance_wheelchair_v5'], label='w_v5')
            ax[0].set_ylabel('')
            ax[0].set_xlabel('Time [s]')
            ax[0].grid()
            ax[0].legend(loc='lower left')
        

            #plt.title(f'Danger comparison with smoothing exp {exp_nr}')
            ax[1].title.set_text('Danger comparison')
            ax[1].plot(df_100.index, df_100['danger_tracker'], label='tracker')
            #ax[1].plot(df_100.index,df_dist_noroll, label='fusion')
            ax[1].plot(df_100.index, df_100['danger_fusion'], label='fusion')
            ax[1].plot(df_100.index, df_100['danger_fusion_smooth'], label='fusion smooth')

            # if cam_d_yolo_FT=='True':
            #     ax[1].plot(df_100.index, df_100['danger_drone_FT'], label='d_FT')
            # if cam_d_yolo_v5=='True':
            #     ax[1].plot(df_100.index, df_100['danger_drone_v5'], label='d_v5')#,marker='o', linestyle=' ')
            # if cam_w_yolo_FT=='True':
            #     ax[1].plot(df_100.index, df_100['danger_wheelchair_FT'], label='w_FT')
            # if cam_w_yolo_v5=='True':
            #     ax[1].plot(df_100.index, df_100['danger_wheelchair_v5'], label='w_v5')
            
            ax[1].set_ylabel('')
            ax[1].set_xlabel('Time [s]')
            ax[1].grid()
            #ax[1].legend(loc="upper right")
            #plot end---------------------_

        #to evaluate from first available data of each sensor 
        first_valid_index = df_100['danger_fusion'].first_valid_index()

        
        #viene rifatto meglio dopo (non considerare)
        #truth = df_100['danger_tracker'].loc[first_valid_index:] < 1
        #decision = df_100['danger_fusion'].loc[first_valid_index:] < 1 #memory of the semaphore

        ## Decision (threshold after DF)
        ### Decision loop
        ### This loop achieves a binary cross/no-cross decision for distance (low-level) fusion

        #considerare questo:
        decision = []
        memory = []
        for i, item in enumerate(df_100['danger_fusion']):
            if item is np.nan:
                item = 1
            if i<10:
                decision.append(0)
                memory.append(0)
            else:
                if sum(memory) + item >= 1:
                    decision.append(1)
                    if item >= 1:
                        memory.append(1)
                    else:
                        memory.append(0)
                    del memory[0]
                else:
                    decision.append(0)
                    memory.append(0)
                    del memory[0]

        #df_100.index = df_100.index - df_100.index[0] #?? cosa succede se lo tolgo?

        #df_100.to_csv('distance_fusion.csv')

        # cast list to pandas format
        decision = pd.Series(decision, index=df_100.index)
        # Make tracker decision binary, cross/no-cross
        truth = df_100['danger_tracker']>1

        # plot real danger function and real decision

        if set_plot=='True':
            #plot start---------------------_
            ax[2].title.set_text('Decision comparison')
            ax[2].plot(df_100.index, df_100['danger_tracker'], label='tracker')
            ax[2].plot(df_100.index, df_100['danger_fusion'], label='fusion')
            ax[2].scatter(df_100.index, truth, label='truth')
            decision = df_100['danger_fusion']>1
            ax[2].scatter(df_100.index, decision, label='decision ')
            ax[2].set_ylabel('')
            ax[2].set_xlabel('Time [s]')
            ax[2].grid()
            ax[2].legend(loc="upper right")
            #plot end---------------------_

        

        #plt.show()
        ## RMSE for distance fusion
        if first_valid_index!= None:
           #print('ok')
            #y_actual = df_100['danger_tracker'].loc[first_valid_index:]
            #y_pred = df_100['danger_fusion'].loc[first_valid_index:]
            y_actual= df_100['danger_tracker']
            y_pred = df_100['danger_fusion']
           #print(y_actual,y_pred)
            MSE = mean_squared_error(y_actual, y_pred)
            rmse = np.sqrt(MSE)
            
            ### Decision metrics
            truth = df_100['danger_tracker']>1
            decision = df_100['danger_fusion']>1
            accuracy = accuracy_score(truth, decision)
            precision = precision_score(truth, decision)
            recall = recall_score(truth, decision)
            f1 = f1_score(truth, decision)
            
        else:
            rmse= None
            f1 = None
            accuracy = None

        RMSE=pd.DataFrame()
        F1_score=pd.DataFrame()
        Accuracy_score=pd.DataFrame()

        if 'fusion' not in RMSE:
            RMSE.insert(0, "fusion", [rmse])
        if 'fusion' not in F1_score:
            F1_score.insert(0, "fusion", [f1])
        if 'fusion' not in Accuracy_score:
            Accuracy_score.insert(0, "fusion", [accuracy])
        
        first_valid_index = df_100['danger_fusion_smooth'].first_valid_index()
        ## RMSE for distance fusion
        if first_valid_index!= None:
            #y_actual = df_100['danger_tracker'].loc[first_valid_index:]
            #y_pred = df_100['danger_fusion'].loc[first_valid_index:]
            y_actual= df_100['danger_tracker']
            y_pred = df_100['danger_fusion_smooth']
           #print(y_actual,y_pred)
            MSE = mean_squared_error(y_actual, y_pred)
            rmse = np.sqrt(MSE)
            
            ### Decision metrics
            truth = df_100['danger_tracker']>1
            decision = df_100['danger_fusion_smooth']>1
            accuracy = accuracy_score(truth, decision)
            precision = precision_score(truth, decision)
            recall = recall_score(truth, decision)
            f1 = f1_score(truth, decision)
            
        else:
            rmse= None
            f1 = None
            accuracy = None

        if 'fusion_smooth' not in RMSE:
            RMSE.insert(0, "fusion_smooth", [rmse])
        if 'fusion_smooth' not in F1_score:
            F1_score.insert(0, "fusion_smooth", [f1])
        if 'fusion_smooth' not in Accuracy_score:
            Accuracy_score.insert(0, "fusion_smooth", [accuracy])
        
        #accuracy, precision, recall, f1        
        #Metrics of single sensors
        ## For SAFECOMP: RMSE for individual sensors
        if range_sens=='True':
            ### RMSE range
            df_100['danger_range'] = df_100.apply(danger_function_composed, args=('range'), axis=1)
            first_valid_index_range = df_100['danger_range'].first_valid_index()
            if first_valid_index_range!= None:
                y_actual = df_100['danger_tracker']#.loc[first_valid_index_range:]
                y_pred = df_100['danger_range']#.loc[first_valid_index_range:]
                MSE = mean_squared_error(y_actual, y_pred)
                rmse = np.sqrt(MSE)
                # Calculate F1-score
                ### This loop achieves a binary cross/no-cross decision for distance (low-level) fusion
                #considerare questo:
                decision = []
                memory = []
                for i, item in enumerate(df_100['danger_range']):
                    if item is np.nan:
                        item = 1
                    if i<10:
                        decision.append(0)
                        memory.append(0)
                    else:
                        if sum(memory) + item >= 1:
                            decision.append(1)
                            if item >= 1:
                                memory.append(1)
                            else:
                                memory.append(0)
                            del memory[0]
                        else:
                            decision.append(0)
                            memory.append(0)
                            del memory[0]

                #df_100.index = df_100.index - df_100.index[0] #?? cosa succede se lo tolgo?
                # cast list to pandas format
                decision = pd.Series(decision, index=df_100.index)
                f1 = f1_score(truth, decision)
                accuracy = accuracy_score(truth, decision)
            else:
                rmse= None
                f1 = None
                accuracy = None

            if 'range_sens' not in RMSE:
                RMSE.insert(0, "range_sens", [rmse])
            if 'range_sens' not in F1_score:
                F1_score.insert(0, "range_sens", [f1])
            if 'range_sens' not in Accuracy_score:
                Accuracy_score.insert(0, "range_sens", [accuracy])



        if cam_w_yolo_FT=='True':
            ### RMSE wheelchair FT
            df_100['danger_wheelchair_FT'] = df_100.apply(danger_function_composed, args=('wheelchair_FT',), axis=1)

            first_valid_index_wheelchair = df_100['danger_wheelchair_FT'].first_valid_index()
            if first_valid_index_wheelchair!= None:

                y_actual = df_100['danger_tracker']#.loc[first_valid_index_wheelchair:]
                y_pred = df_100['danger_wheelchair_FT']#.loc[first_valid_index_wheelchair:]
                MSE = mean_squared_error(y_actual, y_pred)
                rmse = np.sqrt(MSE)

                ### This loop achieves a binary cross/no-cross decision for distance (low-level) fusion
                #considerare questo:
                decision = []
                memory = []
                for i, item in enumerate(df_100['danger_wheelchair_FT']):
                    if item is np.nan:
                        item = 1
                    if i<10:
                        decision.append(0)
                        memory.append(0)
                    else:
                        if sum(memory) + item >= 1:
                            decision.append(1)
                            if item >= 1:
                                memory.append(1)
                            else:
                                memory.append(0)
                            del memory[0]
                        else:
                            decision.append(0)
                            memory.append(0)
                            del memory[0]

                #df_100.index = df_100.index - df_100.index[0] #?? cosa succede se lo tolgo?
                #df_100.to_csv('distance_fusion.csv')
                # cast list to pandas format
                decision = pd.Series(decision, index=df_100.index)
                f1 = f1_score(truth, decision)
                accuracy = accuracy_score(truth, decision)
            else:
                rmse= None
                f1 = None
                accuracy = None
    
            if 'cam_w_yolo_FT' not in RMSE:
                RMSE.insert(0, "cam_w_yolo_FT", [rmse])
            if 'cam_w_yolo_FT' not in F1_score:
                F1_score.insert(0, "cam_w_yolo_FT", [f1])
            if 'cam_w_yolo_FT' not in Accuracy_score:
                Accuracy_score.insert(0, "cam_w_yolo_FT", [accuracy])


        if cam_w_yolo_v5=='True':
            ### RMSE wheelchair v5
            df_100['danger_wheelchair_v5'] = df_100.apply(danger_function_composed, args=('wheelchair_v5',), axis=1)

            first_valid_index_wheelchair = df_100['danger_wheelchair_v5'].first_valid_index()
            if first_valid_index_wheelchair!= None:
                y_actual = df_100['danger_tracker']#.loc[first_valid_index_wheelchair:]
                y_pred = df_100['danger_wheelchair_v5']#.loc[first_valid_index_wheelchair:]
                MSE = mean_squared_error(y_actual, y_pred)
                rmse = np.sqrt(MSE) 

                ### This loop achieves a binary cross/no-cross decision for distance (low-level) fusion
                #considerare questo:
                decision = []
                memory = []
                for i, item in enumerate(df_100['danger_wheelchair_v5']):
                    if item is np.nan:
                        item = 1
                    if i<10:
                        decision.append(0)
                        memory.append(0)
                    else:
                        if sum(memory) + item >= 1:
                            decision.append(1)
                            if item >= 1:
                                memory.append(1)
                            else:
                                memory.append(0)
                            del memory[0]
                        else:
                            decision.append(0)
                            memory.append(0)
                            del memory[0]

                #df_100.index = df_100.index - df_100.index[0] #?? cosa succede se lo tolgo?
                #df_100.to_csv('distance_fusion.csv')
                # cast list to pandas format
                decision = pd.Series(decision, index=df_100.index)
                f1 = f1_score(truth, decision)
                accuracy = accuracy_score(truth, decision)
            else:
                rmse= None
                f1 = None
                accuracy = None
    
            if 'cam_w_yolo_v5' not in RMSE:
                RMSE.insert(0, "cam_w_yolo_v5", [rmse])
            if 'cam_w_yolo_v5' not in F1_score:
                F1_score.insert(0, "cam_w_yolo_v5", [f1])
            if 'cam_w_yolo_v5' not in Accuracy_score:
                Accuracy_score.insert(0, "cam_w_yolo_v5", [accuracy])



        if cam_d_yolo_FT=='True':  
            ### RMSE drone FT
            df_100['danger_drone_FT'] = df_100.apply(danger_function_composed, args=('drone_FT',), axis=1)

            first_valid_index_drone = df_100['danger_drone_FT'].first_valid_index()
            if first_valid_index_drone!= None:
                y_actual = df_100['danger_tracker']#.loc[first_valid_index_drone:]
                y_pred = df_100['danger_drone_FT']#.loc[first_valid_index_drone:]
                MSE = mean_squared_error(y_actual, y_pred)
                rmse = np.sqrt(MSE)
                
                ### This loop achieves a binary cross/no-cross decision for distance (low-level) fusion
                #considerare questo:
                decision = []
                memory = []
                for i, item in enumerate(df_100['danger_drone_FT']):
                    if item is np.nan:
                        item = 1
                    if i<10:
                        decision.append(0)
                        memory.append(0)
                    else:
                        if sum(memory) + item >= 1:
                            decision.append(1)
                            if item >= 1:
                                memory.append(1)
                            else:
                                memory.append(0)
                            del memory[0]
                        else:
                            decision.append(0)
                            memory.append(0)
                            del memory[0]

                #df_100.index = df_100.index - df_100.index[0] #?? cosa succede se lo tolgo?
                #df_100.to_csv('distance_fusion.csv')
                # cast list to pandas format
                decision = pd.Series(decision, index=df_100.index)
                f1 = f1_score(truth, decision)
                accuracy = accuracy_score(truth, decision)
            else:
                rmse= None
                f1 = None
                accuracy = None

            if 'cam_d_yolo_FT' not in RMSE:
                RMSE.insert(0, "cam_d_yolo_FT", [rmse])
            if 'cam_d_yolo_FT' not in F1_score:
                F1_score.insert(0, "cam_d_yolo_FT", [f1])
            if 'cam_d_yolo_FT' not in Accuracy_score:
                Accuracy_score.insert(0, "cam_d_yolo_FT", [accuracy])


        if cam_d_yolo_v5=='True':  
            ### RMSE drone v5
            df_100['danger_drone_v5'] = df_100.apply(danger_function_composed, args=('drone_v5',), axis=1)

            first_valid_index_drone = df_100['danger_drone_v5'].first_valid_index()
            if first_valid_index_drone!= None:
                y_actual = df_100['danger_tracker']#.loc[first_valid_index_drone:]
                y_pred = df_100['danger_drone_v5']#.loc[first_valid_index_drone:]
                #print(first_valid_index_drone)
                #print(y_actual)
                #print(y_pred)
                MSE = mean_squared_error(y_actual, y_pred)
                rmse = np.sqrt(MSE)

                ### This loop achieves a binary cross/no-cross decision for distance (low-level) fusion
                #considerare questo:
                decision = []
                memory = []
                for i, item in enumerate(df_100['danger_drone_v5']):
                    if item is np.nan:
                        item = 1
                    if i<10:
                        decision.append(0)
                        memory.append(0)
                    else:
                        if sum(memory) + item >= 1:
                            decision.append(1)
                            if item >= 1:
                                memory.append(1)
                            else:
                                memory.append(0)
                            del memory[0]
                        else:
                            decision.append(0)
                            memory.append(0)
                            del memory[0]

                #df_100.index = df_100.index - df_100.index[0] #?? cosa succede se lo tolgo?
                #df_100.to_csv('distance_fusion.csv')
                # cast list to pandas format
                decision = pd.Series(decision, index=df_100.index)
                f1 = f1_score(truth, decision)
                accuracy = accuracy_score(truth, decision)
            else:
                rmse= None
                f1 = None
                accuracy = None

            if 'cam_d_yolo_v5' not in RMSE:
                RMSE.insert(0, "cam_d_yolo_v5", [rmse])
            if 'cam_d_yolo_v5' not in F1_score:
                F1_score.insert(0, "cam_d_yolo_v5", [f1])
            if 'cam_d_yolo_v5' not in Accuracy_score:
                Accuracy_score.insert(0, "cam_d_yolo_v5", [accuracy])

        #print('RMSE='+str(RMSE))
        #print('F1_score='+str(F1_score))

        exec(f"RMSE_{filter}= pd.concat([RMSE_{filter}, RMSE], ignore_index=True)")

        exec(f"F1_score_{filter}= pd.concat([F1_score_{filter}, F1_score], ignore_index=True)")
        exec(f"Accuracy_score_{filter}= pd.concat([Accuracy_score_{filter}, Accuracy_score], ignore_index=True)")

        #exec(f"print('RMSE_{filter}='+str(RMSE_{filter}))")
        #exec(f"print('F1_score_{filter}='+str(F1_score_{filter}))")
        #exec(f"print('Accuracy_score_{filter}='+str(Accuracy_score_{filter}))")
    
    exec(f"RMSE_{filter}_mean= RMSE_{filter}.mean()")
    exec(f"RMSE_{filter}_std=  RMSE_{filter}.std()")
    exec(f"F1_score_{filter}_mean= F1_score_{filter}.mean()*100")
    exec(f"F1_score_{filter}_std= F1_score_{filter}.std()*100")
    exec(f"Accuracy_score_{filter}_mean= Accuracy_score_{filter}.mean()*100")
    exec(f"Accuracy_score_{filter}_std= Accuracy_score_{filter}.std()*100")
    
    
    exec(f"F1values= pd.concat([F1values, F1_score_{filter}],axis=1,ignore_index=False)")
    if filter=='original':
        F1values=F1values.rename(columns={'fusion': 'fusion_new_original'})
    elif filter=='fog_1':
        F1values=F1values.rename(columns={'fusion': 'fusion_new_1'})
    elif filter=='fog_2':
        F1values=F1values.rename(columns={'fusion': 'fusion_new_fog_2'})
    elif filter=='rain_1':
        F1values=F1values.rename(columns={'fusion': 'fusion_new_rain_1'})
    elif filter=='rain_2':
        F1values=F1values.rename(columns={'fusion': 'fusion_new_rain_2'})
    if filter=='bright_2':
        F1values=F1values.rename(columns={'fusion': 'fusion_new_bright_2'})
    if filter=='dark_2':
        F1values=F1values.rename(columns={'fusion': 'fusion_new_dark_2'})

    exec(f"RMSEvalues= pd.concat([RMSEvalues, RMSE_{filter}],axis=1,ignore_index=False)")
    if filter=='original':
        RMSEvalues=RMSEvalues.rename(columns={'fusion': 'fusion_new_original'})
    elif filter=='fog_1':
        RMSEvalues=RMSEvalues.rename(columns={'fusion': 'fusion_new_1'})
    elif filter=='fog_2':
        RMSEvalues=RMSEvalues.rename(columns={'fusion': 'fusion_new_fog_2'})
    elif filter=='rain_1':
        RMSEvalues=RMSEvalues.rename(columns={'fusion': 'fusion_new_rain_1'})
    elif filter=='rain_2':
        RMSEvalues=RMSEvalues.rename(columns={'fusion': 'fusion_new_rain_2'})
    if filter=='bright_2':
        RMSEvalues=RMSEvalues.rename(columns={'fusion': 'fusion_new_bright_2'})
    if filter=='dark_2':
        RMSEvalues=RMSEvalues.rename(columns={'fusion': 'fusion_new_dark_2'})

    exec(f"ACCvalues= pd.concat([ACCvalues, Accuracy_score_{filter}],axis=1,ignore_index=False)")
    if filter=='original':
        ACCvalues=ACCvalues.rename(columns={'fusion': 'fusion_new_original'})
    elif filter=='fog_1':
        ACCvalues=ACCvalues.rename(columns={'fusion': 'fusion_new_1'})
    elif filter=='fog_2':
        ACCvalues=ACCvalues.rename(columns={'fusion': 'fusion_new_fog_2'})
    elif filter=='rain_1':
        ACCvalues=ACCvalues.rename(columns={'fusion': 'fusion_new_rain_1'})
    elif filter=='rain_2':
        ACCvalues=ACCvalues.rename(columns={'fusion': 'fusion_new_rain_2'})
    if filter=='bright_2':
        ACCvalues=ACCvalues.rename(columns={'fusion': 'fusion_new_bright_2'})
    if filter=='dark_2':
        ACCvalues=ACCvalues.rename(columns={'fusion': 'fusion_new_dark_2'})

F1values = F1values.drop(['cam_d_yolo_v5'], axis=1)
F1values = F1values.drop(['cam_d_yolo_FT'], axis=1)
F1values = F1values.drop(['cam_w_yolo_v5'], axis=1)
F1values = F1values.drop(['cam_w_yolo_FT'], axis=1)
F1values = F1values.drop(['fusion_smooth'], axis=1)
folder_name='safe_road_data/'
F1values.to_csv(f'{folder_name}/F1values_fusion_new.csv')
    
    




In [56]:
F1values

,fusion_new_original,fusion_new_bright_2,fusion_new_bright_2,fusion_new_dark_2,fusion_new_dark_2,fusion_new_dark_2,fusion_new_1,fusion_new_1,fusion_new_fog_2,fusion_new_rain_1,fusion_new_rain_1,fusion_new_rain_2,fusion
0,0.171233,0.195946,0.199262,0.183051,0.250980,0.287037,0.197531,0.256000,0.343195,0.0,0.253968,0.046875,0.000000
1,0.645299,0.645299,0.737052,0.716599,0.644068,0.574610,0.617711,0.594235,0.291105,0.0,0.621849,0.403023,0.135294
2,0.640669,0.651934,0.681081,0.691892,0.577259,0.628571,0.651685,0.616715,0.279570,0.0,0.620690,0.394649,0.032258
3,0.637615,0.616092,0.610092,0.580189,0.625000,0.605634,0.608899,0.502488,0.258065,0.0,0.668161,0.299435,0.006689
4,0.456140,0.486022,0.525292,0.550562,0.454352,0.414579,0.471366,0.453159,0.288714,0.0,0.533333,0.333333,0.102564
5,0.486322,0.566474,0.591781,0.627968,0.389937,0.361371,0.421687,0.462006,0.204225,0.0,0.519774,0.457831,0.055866


In [ ]:
ACCvalues

In [ ]:
RMSEvalues

# Preparation for the table of ITSC paper

### preparation for RMSE table in overleaf


In [ ]:
#print("%.2f" %RMSE_original_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_original_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_fog_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_fog_1_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_fog_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_fog_2_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_fog_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_fog_3_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_rain_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_rain_1_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_rain_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_rain_2_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_rain_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_rain_3_std['cam_d_yolo_v5'], ')\\',"\\")
#print('&&Y8&', "%.2f" %RMSE_original_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_original_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_fog_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_fog_1_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_fog_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_fog_2_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_fog_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_fog_3_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_rain_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_rain_1_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_rain_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_rain_2_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_rain_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_rain_3_std['cam_d_yolo_FT'], ')\\',"\\")
#print('----------------------------------')
#print("%.2f" %RMSE_original_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_original_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_fog_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_fog_1_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_fog_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_fog_2_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_fog_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_fog_3_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_rain_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_rain_1_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_rain_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_rain_2_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_rain_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_rain_3_std['cam_w_yolo_v5'], ')\\',"\\")
#print('&&Y8&',"%.2f" %RMSE_original_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_original_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_fog_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_fog_1_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_fog_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_fog_2_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_fog_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_fog_3_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_rain_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_rain_1_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_rain_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_rain_2_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_rain_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_rain_3_std['cam_w_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %RMSE_original_mean['fusion'],'(' ,"%.2f" %RMSE_original_std['fusion'], ')&',"%.2f" %RMSE_fog_1_mean['fusion'],'(' ,"%.2f" %RMSE_fog_1_std['fusion'], ')&',"%.2f" %RMSE_fog_2_mean['fusion'],'(' ,"%.2f" %RMSE_fog_2_std['fusion'], ')&',"%.2f" %RMSE_fog_3_mean['fusion'],'(' ,"%.2f" %RMSE_fog_3_std['fusion'], ')&',"%.2f" %RMSE_rain_1_mean['fusion'],'(' ,"%.2f" %RMSE_rain_1_std['fusion'], ')&',"%.2f" %RMSE_rain_2_mean['fusion'],'(' ,"%.2f" %RMSE_rain_2_std['fusion'], ')&',"%.2f" %RMSE_rain_3_mean['fusion'],'(' ,"%.2f" %RMSE_rain_3_std['fusion'], ')\\',"\\")
#print('----------------------------------')
#print("%.2f" %RMSE_original_mean['fusion_smooth'],'(' ,"%.2f" %RMSE_original_std['fusion_smooth'], ')&',"%.2f" %RMSE_fog_1_mean['fusion_smooth'],'(' ,"%.2f" %RMSE_fog_1_std['fusion_smooth'], ')&',"%.2f" %RMSE_fog_2_mean['fusion_smooth'],'(' ,"%.2f" %RMSE_fog_2_std['fusion_smooth'], ')&',"%.2f" %RMSE_fog_3_mean['fusion_smooth'],'(' ,"%.2f" %RMSE_fog_3_std['fusion_smooth'], ')&',"%.2f" %RMSE_rain_1_mean['fusion_smooth'],'(' ,"%.2f" %RMSE_rain_1_std['fusion_smooth'], ')&',"%.2f" %RMSE_rain_2_mean['fusion_smooth'],'(' ,"%.2f" %RMSE_rain_2_std['fusion_smooth'], ')&',"%.2f" %RMSE_rain_3_mean['fusion_smooth'],'(' ,"%.2f" %RMSE_rain_3_std['fusion_smooth'], ')\\',"\\")

# RMSE Dark e brigth

In [ ]:
# print("%.2f" %RMSE_original_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_original_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_bright_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_bright_1_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_bright_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_bright_2_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_bright_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_bright_3_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_dark_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_dark_1_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_dark_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_dark_2_std['cam_d_yolo_v5'], ')&',"%.2f" %RMSE_dark_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %RMSE_dark_3_std['cam_d_yolo_v5'], ')\\',"\\")
# print('&&Y8&', "%.2f" %RMSE_original_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_original_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_bright_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_bright_1_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_bright_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_bright_2_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_bright_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_bright_3_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_dark_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_dark_1_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_dark_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_dark_2_std['cam_d_yolo_FT'], ')&',"%.2f" %RMSE_dark_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %RMSE_dark_3_std['cam_d_yolo_FT'], ')\\',"\\")
# print('----------------------------------')
# print("%.2f" %RMSE_original_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_original_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_bright_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_bright_1_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_bright_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_bright_2_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_bright_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_bright_3_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_dark_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_dark_1_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_dark_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_dark_2_std['cam_w_yolo_v5'], ')&',"%.2f" %RMSE_dark_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %RMSE_dark_3_std['cam_w_yolo_v5'], ')\\',"\\")
# print('&&Y8&',"%.2f" %RMSE_original_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_original_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_bright_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_bright_1_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_bright_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_bright_2_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_bright_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_bright_3_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_dark_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_dark_1_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_dark_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_dark_2_std['cam_w_yolo_FT'], ')&',"%.2f" %RMSE_dark_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %RMSE_dark_3_std['cam_w_yolo_FT'], ')\\',"\\")
# print('----------------------------------')
print("%.2f" %RMSE_original_mean['fusion'],'(' ,"%.2f" %RMSE_original_std['fusion'], ')&',"%.2f" %RMSE_bright_1_mean['fusion'],'(' ,"%.2f" %RMSE_bright_1_std['fusion'], ')&',"%.2f" %RMSE_bright_2_mean['fusion'],'(' ,"%.2f" %RMSE_bright_2_std['fusion'], ')&',"%.2f" %RMSE_bright_3_mean['fusion'],'(' ,"%.2f" %RMSE_bright_3_std['fusion'], ')&',"%.2f" %RMSE_dark_1_mean['fusion'],'(' ,"%.2f" %RMSE_dark_1_std['fusion'], ')&',"%.2f" %RMSE_dark_2_mean['fusion'],'(' ,"%.2f" %RMSE_dark_2_std['fusion'], ')&',"%.2f" %RMSE_dark_3_mean['fusion'],'(' ,"%.2f" %RMSE_dark_3_std['fusion'], ')\\',"\\")

# F1 fog and rain

In [ ]:
# print("%.2f" %F1_score_original_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_original_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_fog_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_fog_1_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_fog_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_fog_2_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_fog_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_fog_3_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_rain_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_rain_1_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_rain_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_rain_2_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_rain_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_rain_3_std['cam_d_yolo_v5'], ')\\',"\\")
# print('&&Y8&', "%.2f" %F1_score_original_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_original_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_fog_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_fog_1_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_fog_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_fog_2_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_fog_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_fog_3_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_rain_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_rain_1_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_rain_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_rain_2_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_rain_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_rain_3_std['cam_d_yolo_FT'], ')\\',"\\")
# print('----------------------------------')
# print("%.2f" %F1_score_original_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_original_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_fog_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_fog_1_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_fog_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_fog_2_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_fog_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_fog_3_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_rain_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_rain_1_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_rain_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_rain_2_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_rain_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_rain_3_std['cam_w_yolo_v5'], ')\\',"\\")
# print('&&Y8&',"%.2f" %F1_score_original_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_original_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_fog_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_fog_1_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_fog_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_fog_2_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_fog_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_fog_3_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_rain_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_rain_1_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_rain_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_rain_2_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_rain_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_rain_3_std['cam_w_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %F1_score_original_mean['fusion'],'(' ,"%.2f" %F1_score_original_std['fusion'], ')&',"%.2f" %F1_score_fog_1_mean['fusion'],'(' ,"%.2f" %F1_score_fog_1_std['fusion'], ')&',"%.2f" %F1_score_fog_2_mean['fusion'],'(' ,"%.2f" %F1_score_fog_2_std['fusion'], ')&',"%.2f" %F1_score_fog_3_mean['fusion'],'(' ,"%.2f" %F1_score_fog_3_std['fusion'], ')&',"%.2f" %F1_score_rain_1_mean['fusion'],'(' ,"%.2f" %F1_score_rain_1_std['fusion'], ')&',"%.2f" %F1_score_rain_2_mean['fusion'],'(' ,"%.2f" %F1_score_rain_2_std['fusion'], ')&',"%.2f" %F1_score_rain_3_mean['fusion'],'(' ,"%.2f" %F1_score_rain_3_std['fusion'], ')\\',"\\")
#print('----------------------------------')
#print("%.2f" %F1_score_original_mean['fusion_smooth'],'(' ,"%.2f" %F1_score_original_std['fusion_smooth'], ')&',"%.2f" %F1_score_fog_1_mean['fusion_smooth'],'(' ,"%.2f" %F1_score_fog_1_std['fusion_smooth'], ')&',"%.2f" %F1_score_fog_2_mean['fusion_smooth'],'(' ,"%.2f" %F1_score_fog_2_std['fusion_smooth'], ')&',"%.2f" %F1_score_fog_3_mean['fusion_smooth'],'(' ,"%.2f" %F1_score_fog_3_std['fusion_smooth'], ')&',"%.2f" %F1_score_rain_1_mean['fusion_smooth'],'(' ,"%.2f" %F1_score_rain_1_std['fusion_smooth'], ')&',"%.2f" %F1_score_rain_2_mean['fusion_smooth'],'(' ,"%.2f" %F1_score_rain_2_std['fusion_smooth'], ')&',"%.2f" %F1_score_rain_3_mean['fusion_smooth'],'(' ,"%.2f" %F1_score_rain_3_std['fusion_smooth'], ')\\',"\\")

# F1 Dark e brigth

In [ ]:
# print("%.2f" %F1_score_original_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_original_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_bright_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_bright_1_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_bright_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_bright_2_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_bright_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_bright_3_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_dark_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_dark_1_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_dark_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_dark_2_std['cam_d_yolo_v5'], ')&',"%.2f" %F1_score_dark_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %F1_score_dark_3_std['cam_d_yolo_v5'], ')\\',"\\")
# print('&&Y8&', "%.2f" %F1_score_original_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_original_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_bright_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_bright_1_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_bright_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_bright_2_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_bright_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_bright_3_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_dark_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_dark_1_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_dark_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_dark_2_std['cam_d_yolo_FT'], ')&',"%.2f" %F1_score_dark_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %F1_score_dark_3_std['cam_d_yolo_FT'], ')\\',"\\")
# print('----------------------------------')
# print("%.2f" %F1_score_original_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_original_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_bright_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_bright_1_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_bright_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_bright_2_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_bright_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_bright_3_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_dark_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_dark_1_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_dark_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_dark_2_std['cam_w_yolo_v5'], ')&',"%.2f" %F1_score_dark_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %F1_score_dark_3_std['cam_w_yolo_v5'], ')\\',"\\")
# print('&&Y8&',"%.2f" %F1_score_original_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_original_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_bright_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_bright_1_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_bright_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_bright_2_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_bright_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_bright_3_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_dark_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_dark_1_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_dark_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_dark_2_std['cam_w_yolo_FT'], ')&',"%.2f" %F1_score_dark_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %F1_score_dark_3_std['cam_w_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %F1_score_original_mean['fusion'],'(' ,"%.2f" %F1_score_original_std['fusion'], ')&',"%.2f" %F1_score_bright_1_mean['fusion'],'(' ,"%.2f" %F1_score_bright_1_std['fusion'], ')&',"%.2f" %F1_score_bright_2_mean['fusion'],'(' ,"%.2f" %F1_score_bright_2_std['fusion'], ')&',"%.2f" %F1_score_bright_3_mean['fusion'],'(' ,"%.2f" %F1_score_bright_3_std['fusion'], ')&',"%.2f" %F1_score_dark_1_mean['fusion'],'(' ,"%.2f" %F1_score_dark_1_std['fusion'], ')&',"%.2f" %F1_score_dark_2_mean['fusion'],'(' ,"%.2f" %F1_score_dark_2_std['fusion'], ')&',"%.2f" %F1_score_dark_3_mean['fusion'],'(' ,"%.2f" %F1_score_dark_3_std['fusion'], ')\\',"\\")

### preparation for Accuracy_score table in overleaf


In [ ]:
# print("%.2f" %Accuracy_score_original_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_original_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_fog_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_fog_1_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_fog_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_fog_2_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_fog_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_fog_3_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_rain_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_rain_1_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_rain_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_rain_2_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_rain_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_rain_3_std['cam_d_yolo_v5'], ')\\',"\\")
# print('&&Y8&', "%.2f" %Accuracy_score_original_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_original_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_fog_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_fog_1_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_fog_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_fog_2_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_fog_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_fog_3_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_rain_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_rain_1_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_rain_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_rain_2_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_rain_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_rain_3_std['cam_d_yolo_FT'], ')\\',"\\")
# print('----------------------------------')
# print("%.2f" %Accuracy_score_original_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_original_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_fog_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_fog_1_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_fog_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_fog_2_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_fog_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_fog_3_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_rain_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_rain_1_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_rain_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_rain_2_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_rain_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_rain_3_std['cam_w_yolo_v5'], ')\\',"\\")
# print('&&Y8&',"%.2f" %Accuracy_score_original_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_original_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_fog_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_fog_1_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_fog_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_fog_2_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_fog_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_fog_3_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_rain_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_rain_1_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_rain_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_rain_2_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_rain_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_rain_3_std['cam_w_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %Accuracy_score_original_mean['fusion'],'(' ,"%.2f" %Accuracy_score_original_std['fusion'], ')&',"%.2f" %Accuracy_score_fog_1_mean['fusion'],'(' ,"%.2f" %Accuracy_score_fog_1_std['fusion'], ')&',"%.2f" %Accuracy_score_fog_2_mean['fusion'],'(' ,"%.2f" %Accuracy_score_fog_2_std['fusion'], ')&',"%.2f" %Accuracy_score_fog_3_mean['fusion'],'(' ,"%.2f" %Accuracy_score_fog_3_std['fusion'], ')&',"%.2f" %Accuracy_score_rain_1_mean['fusion'],'(' ,"%.2f" %Accuracy_score_rain_1_std['fusion'], ')&',"%.2f" %Accuracy_score_rain_2_mean['fusion'],'(' ,"%.2f" %Accuracy_score_rain_2_std['fusion'], ')&',"%.2f" %Accuracy_score_rain_3_mean['fusion'],'(' ,"%.2f" %Accuracy_score_rain_3_std['fusion'], ')\\',"\\")
#print('----------------------------------')
#print("%.2f" %Accuracy_score_original_mean['fusion_smooth'],'(' ,"%.2f" %Accuracy_score_original_std['fusion_smooth'], ')&',"%.2f" %Accuracy_score_fog_1_mean['fusion_smooth'],'(' ,"%.2f" %Accuracy_score_fog_1_std['fusion_smooth'], ')&',"%.2f" %Accuracy_score_fog_2_mean['fusion_smooth'],'(' ,"%.2f" %Accuracy_score_fog_2_std['fusion_smooth'], ')&',"%.2f" %Accuracy_score_fog_3_mean['fusion_smooth'],'(' ,"%.2f" %Accuracy_score_fog_3_std['fusion_smooth'], ')&',"%.2f" %Accuracy_score_rain_1_mean['fusion_smooth'],'(' ,"%.2f" %Accuracy_score_rain_1_std['fusion_smooth'], ')&',"%.2f" %Accuracy_score_rain_2_mean['fusion_smooth'],'(' ,"%.2f" %Accuracy_score_rain_2_std['fusion_smooth'], ')&',"%.2f" %Accuracy_score_rain_3_mean['fusion_smooth'],'(' ,"%.2f" %Accuracy_score_rain_3_std['fusion_smooth'], ')\\',"\\")

# Accuracy Dark e brigth

In [ ]:
# print("%.2f" %Accuracy_score_original_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_original_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_bright_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_bright_1_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_bright_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_bright_2_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_bright_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_bright_3_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_dark_1_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_dark_1_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_dark_2_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_dark_2_std['cam_d_yolo_v5'], ')&',"%.2f" %Accuracy_score_dark_3_mean['cam_d_yolo_v5'],'(' ,"%.2f" %Accuracy_score_dark_3_std['cam_d_yolo_v5'], ')\\',"\\")
# print('&&Y8&', "%.2f" %Accuracy_score_original_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_original_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_bright_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_bright_1_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_bright_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_bright_2_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_bright_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_bright_3_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_dark_1_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_dark_1_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_dark_2_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_dark_2_std['cam_d_yolo_FT'], ')&',"%.2f" %Accuracy_score_dark_3_mean['cam_d_yolo_FT'],'(' ,"%.2f" %Accuracy_score_dark_3_std['cam_d_yolo_FT'], ')\\',"\\")
# print('----------------------------------')
# print("%.2f" %Accuracy_score_original_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_original_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_bright_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_bright_1_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_bright_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_bright_2_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_bright_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_bright_3_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_dark_1_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_dark_1_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_dark_2_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_dark_2_std['cam_w_yolo_v5'], ')&',"%.2f" %Accuracy_score_dark_3_mean['cam_w_yolo_v5'],'(' ,"%.2f" %Accuracy_score_dark_3_std['cam_w_yolo_v5'], ')\\',"\\")
# print('&&Y8&',"%.2f" %Accuracy_score_original_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_original_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_bright_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_bright_1_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_bright_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_bright_2_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_bright_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_bright_3_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_dark_1_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_dark_1_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_dark_2_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_dark_2_std['cam_w_yolo_FT'], ')&',"%.2f" %Accuracy_score_dark_3_mean['cam_w_yolo_FT'],'(' ,"%.2f" %Accuracy_score_dark_3_std['cam_w_yolo_FT'], ')\\',"\\")
print('----------------------------------')
print("%.2f" %Accuracy_score_original_mean['fusion'],'(' ,"%.2f" %Accuracy_score_original_std['fusion'], ')&',"%.2f" %Accuracy_score_bright_1_mean['fusion'],'(' ,"%.2f" %Accuracy_score_bright_1_std['fusion'], ')&',"%.2f" %Accuracy_score_bright_2_mean['fusion'],'(' ,"%.2f" %Accuracy_score_bright_2_std['fusion'], ')&',"%.2f" %Accuracy_score_bright_3_mean['fusion'],'(' ,"%.2f" %Accuracy_score_bright_3_std['fusion'], ')&',"%.2f" %Accuracy_score_dark_1_mean['fusion'],'(' ,"%.2f" %Accuracy_score_dark_1_std['fusion'], ')&',"%.2f" %Accuracy_score_dark_2_mean['fusion'],'(' ,"%.2f" %Accuracy_score_dark_2_std['fusion'], ')&',"%.2f" %Accuracy_score_dark_3_mean['fusion'],'(' ,"%.2f" %Accuracy_score_dark_3_std['fusion'], ')\\',"\\")